<a href="https://colab.research.google.com/github/zenAurelius/HRAI3/blob/main/notebooks/test_base3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras import layers

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/HRAI/data

In [ ]:
dfraw = pd.read_csv('pmu2017_os.zip')
list(dfraw.columns)

In [ ]:
# prompt: je veux une concatenation de pff_tendanceReference_1 et pff_tendanceReference_2 mais en remplacant dans chaque colonne 0 par + 0.5 par = et 1 par -

def replace_values(value):
  if value == 0:
    return '+'
  elif value == 0.5:
    return '='
  elif value == 1:
    return '-'
  else:
    return value

dfraw['pfs_dTendanceReference'] = (
    dfraw['pff_tendanceReference_1'].apply(replace_values).astype(str) +
    dfraw['pff_tendanceReference_2'].apply(replace_values).astype(str)
)

dfraw['pfs_dTendanceReference']


In [ ]:
dfraw['pfs_dTendanceReference'].value_counts()

In [ ]:
# prompt: je veux la proportion de tgf_win_1 = 1 pour chaque valeur possible de dfraw['pfs_dTendanceReference']

# Group by 'pfs_dTendanceReference' and calculate the proportion of 'tgf_win_1' equal to 1
proportion_tgf_win_1 = dfraw.groupby('pfs_dTendanceReference')['tgf_win_1'].mean()

proportion_tgf_win_1


In [ ]:
dfraw = dfraw[~dfraw.pff_rapportDirect_1.isna()].copy()
dfraw['win_cote'] = (dfraw.pff_rapportDirect_1 < dfraw.pff_rapportDirect_2).astype(int)
dfraw[['win_cote','pff_rapportDirect_1', 'pff_rapportDirect_2']]
dfraw['pff_dOrd'] = dfraw.pff_ord_1 - dfraw.pff_ord_2
dfraw['pff_dMuC'] = dfraw.pff_mu_pis_cheval_1 - dfraw.pff_mu_pis_cheval_2
dfraw['pff_dMuD'] = dfraw.pff_mu_pis_driver_1 - dfraw.pff_mu_pis_driver_2

In [ ]:
def complete(df, cols):
  cols_complete = []
  for b in (cols):
    if(b.endswith('_')):
      cols_complete.append(b + '1')
      cols_complete.append(b + '2')
    else:
      cols_complete.append(b)
  return cols_complete

def calc_ratio(df, ratio, num_features):
    for col in ratio:
      col1 = col + '1'
      col2 = col + '2'
      if col2 in df.columns:
        df[f'{col}ratio'] = (df[col1] / (df[col1] + df[col2])).replace([np.inf, -np.inf, np.nan], 0.5).fillna(0.5)
        df[f'win_{col}'] = (df[f'{col}ratio'] > 0.5).astype(int)
        num_features.append(f'{col}ratio')
        #num_features.append(col1)
    return df


# INIT FEATURES/TARGET/CONST
NONFEATURES = ['aid_cr', 'pis_cheval_', 'tgi_ordreArrivee_', 'pff_rapportDirect_1', 'tgb_estGagnant_1', 'tgb_estPlace_1', 'win_cote']
NONFEATURES = complete(dfraw, NONFEATURES)
# Les features liées à la cotes : 'pff_normcote_','pff_rapportDirect_','pff_rapportReference_',
RATIO_F = ['pfi_chAge_', 'pfi_chNbCourses_','pfi_chNbVictoires_','pfi_chNbPlaces_','pfi_chNbSecond_','pfi_chNbTroisieme_','pff_chGainTotal_','pff_chGainVictoire_','pff_chGainPlace_','pff_chGainAnnee_','pff_chGainAnneePrec_','pff_handicap_','pfi_placeCorde_','pff_txReclamation_']
#RATIO_F = []
NUM_FEATURES = ['rfi_nbPartants','rff_temperature','rfi_ventForce','pfb_chInedit_1','pff_dOrd']
#NUM_FEATURES = ['pff_dMuC','pff_dMuD','pff_sgsq_1','pff_sgsq_2']
CAT_FEATURES = ['pfs_dSexe', 'pfs_dTendanceReference']
#CAT_FEATURES = []
CAT_FEATURES_VOCAB = {
    'pfs_dSexe': ['FF', 'FM', 'FH', 'HF', 'HM', 'HH', 'MM', 'MF', 'MH'],
    'pfs_dTendanceReference': ['++','+=','+-','=+','==','=-','-+','-=','--']
}
dfraw = calc_ratio(dfraw, RATIO_F, NUM_FEATURES)
FEATURES = NUM_FEATURES + CAT_FEATURES


BATCH_SIZE = 256
MT_REG = 'regression'
MT_CLA = 'classification'
MODEL_TYPE = MT_REG
if MODEL_TYPE == 'classification':
  TARGET = ['tgf_win_1']
  NONFEATURES.append('tgf_pwin_1')
else:
  TARGET = ['tgf_pwin_1']
  NONFEATURES.append('tgf_win_1')
ALL_COLS = FEATURES + TARGET

df = dfraw[ALL_COLS + NONFEATURES].copy()
df

In [ ]:
# SPLIT TRAIN, TEST, VAL

limit_train = df['aid_cr'].iloc[int(0.8 * len(df))]
limit_val = df['aid_cr'].iloc[int(0.9 * len(df))]
print(limit_train, limit_val)
train = df[df.aid_cr < limit_train].copy()
val = df[(df.aid_cr >= limit_train) & (df.aid_cr < limit_val)].copy()
test = df[df.aid_cr >= limit_val].copy()
print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train[NUM_FEATURES] = scaler.fit_transform(train[NUM_FEATURES])
val[NUM_FEATURES] = scaler.transform(val[NUM_FEATURES])
test[NUM_FEATURES] = scaler.transform(test[NUM_FEATURES])

In [ ]:
# CONVERSIONS INPUT

# DF TO DATASET
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  df = dataframe[ALL_COLS].copy()
  labels = df.pop(TARGET[0])
  df = {key: np.array(value)[:,tf.newaxis] for key, value in df.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

# GET INPUT LAYER
def get_input_layer(name, dataset):
  print(name)
  # normalizer = layers.Normalization(axis=None)
  # Récupère un dataset avec seulement les colonnes passées dans 'name'
  #feature_ds = dataset.map(lambda x, y: x[name])
  #print(feature_ds)
  #layers.Input(
  return layers.Input( name=name, shape=(), dtype="float32" )
  #normalizer.adapt(feature_ds)
  #return normalizer

# GET NORMALIZATION LAYER
def get_normalization_layer(name, dataset):
  print(name)
  normalizer = layers.Normalization(axis=None)
  # Récupère un dataset avec seulement les colonnes passées dans 'name'
  feature_ds = dataset.map(lambda x, y: x[name])
  print(feature_ds)
  normalizer.adapt(feature_ds)
  return normalizer

# GET CATEGORY ENCODING LAYER
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Lookup Layer, soit StringLookup, soit IntergerLookup
  if dtype == 'string':
    vocabulary = CAT_FEATURES_VOCAB[name]
    index = layers.StringLookup(
                vocabulary=vocabulary,
                mask_token=None,
                num_oov_indices=0,
                output_mode="binary"
            )
  else:
    index = layers.IntegerLookup(max_tokens=max_tokens)

  return index

In [ ]:
# TO DATASET

train_ds = df_to_dataset(train, shuffle=False, batch_size=BATCH_SIZE)
val_ds = df_to_dataset(val, shuffle=False, batch_size=BATCH_SIZE)
test_ds = df_to_dataset(test, shuffle=False, batch_size=BATCH_SIZE)

In [ ]:
all_inputs = {}
encoded_features = []
import keras

#for header in NUM_FEATURES:
for header in NUM_FEATURES:
  numeric_col = tf.keras.Input(shape=(), name=header, dtype='float32')
  #normalization_layer = get_normalization_layer(header, train_ds)
  #encoded_numeric_col = normalization_layer(numeric_col)
  encoded_numeric_col = keras.ops.expand_dims(numeric_col,-1)
  all_inputs[header] = numeric_col
  encoded_features.append(encoded_numeric_col)

for header in CAT_FEATURES:
  categorical_col = tf.keras.Input(shape=(), name=header, dtype='string')
  lookup = get_category_encoding_layer(header, dataset=train_ds, dtype='string', max_tokens=9)
  encoded_feature = lookup( keras.ops.expand_dims(categorical_col, -1))
  all_inputs[header] = categorical_col
  encoded_features.append(encoded_feature)

In [ ]:
# MODEL REGRESSION
all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(128, activation="relu")(all_features)
x = tf.keras.layers.Dense(32, activation="relu")(x)
output = tf.keras.layers.Dense(1)(x)
print(encoded_features)

model = tf.keras.Model(all_inputs, output)
if(MODEL_TYPE == MT_REG):
  model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(0.001))
else:
  model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy"],
              run_eagerly=True)

In [ ]:
# Use `rankdir='LR'` to make the graph horizontal.
tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")


In [ ]:
model.fit(train_ds, epochs=4, validation_data=val_ds)

In [ ]:
def predict(model, ds, df, pcol):
  predictions = model.predict(ds)
  df[pcol] = predictions
  threshold = 0.5
  df[f'{pcol}_w'] = (predictions > threshold).astype(int)

  # Le reste pourrait être ailleurs
def prepare_comparaison(df):
  df['win1'] = (df.tgf_pwin_1 > 0.5).astype(int)

  #df['win_ord'] = (df.pff_dOrd > 0.5).astype(int)
  #df['win_gain'] = (df.pff_dGain > 0.5).astype(int)

def eval(df, pcol):
  print('positifs')
  print(len(df[(df[f'{pcol}_w'] == 1) & (df.win1 == 1)]) / len(df[(df.win1 == 1)]))
  print(len(df[(df.win_cote == 1) & (df.win1 == 1)]) / len(df[(df.win1 == 1)]))
  #print(len(df[(df.win_ord == 1) & (df.win1 == 1)]) / len(df[(df.win1 == 1)]))
  print('negatifs')
  print(len(df[(df[f'{pcol}_w'] == 0) & (df.win1 == 0)]) / len(df[(df.win1 == 0)]))
  print(len(df[(df.win_cote == 0) & (df.win1 == 0)]) / len(df[(df.win1 == 0)]))
  #print(len(df[(df.win_ord == 0) & (df.win1 == 0)]) / len(df[(df.win1 == 0)]))

In [ ]:
predict(model, train_ds, train, 'pred')
prepare_comparaison(train)
eval(train, 'pred')

In [ ]:
predict(model, test_ds, test, 'pred')
prepare_comparaison(test)
eval(test, 'pred')

In [ ]:
def pct_place(df):
    n = len(df)
    npl = len(df[df.tgb_estPlace_1])
    nga = len(df[df.tgb_estGagnant_1])
    print(f'placé {100 * npl / n:.2f}%  [{npl}/{n}] - gagnant {100 * nga / n:.2f}% [{nga}/{n}]')
    df['gain_gagnant'] = np.where(df['tgb_estGagnant_1'], df['pff_rapportDirect_1']-1, -1)
    print(f'gain gagnant = {df.gain_gagnant.sum()}')

In [ ]:
result = test.groupby(['aid_cr', 'pis_cheval_1']).first().reset_index()
bycr = result.groupby('aid_cr')
result['pred_R'] = bycr['pred'].rank(ascending=False)
pct_place(result[result['pred_R'] == 1.0].copy())
pct_place(result[result['pred_R'] == 2.0].copy())
pct_place(result[result['pred_R'] == 3.0].copy())
result['cote_R'] = bycr['pff_rapportDirect_1'].rank(ascending=True)
pct_place(result[result['cote_R'] == 1.0].copy())
pct_place(result[result['cote_R'] == 2.0].copy())
pct_place(result[result['cote_R'] == 3.0].copy())

**TESTS**

In [ ]:
dfraw[dfraw.pff_ord__ratio > 10][['pff_ord__ratio','pff_ord_1','pff_ord_2']]

In [ ]:
# prompt: je veux un graph en nuage de point montrant la relation entre dfraw.pff_ord_ratio et defraw.tgf_win_1

import matplotlib.pyplot as plt

df = dfraw.tail(1000).copy()
df['d_ord'] = df.pff_ord_1 - df.pff_ord_2
print(df.d_ord.value_counts())

# Create a scatter plot
plt.scatter(df.d_ord, df.tgf_pwin_1)

# Add labels and title
plt.xlabel('dfraw.pff_ord_ratio')
plt.ylabel('dfraw.tgf_win_1')
plt.title('Relationship between pff_ord_ratio and tgf_win_1')

# Show the plot
plt.show()


In [ ]:
dfraw.pff_favorisReference_1.value_counts()

In [ ]:
# prompt: Pour toutes les colonnes qui commencent par 'pfi' ou 'pff' et finissent par '1' du dataframe dfraw, je voudrais calculer le rapport de la colonne qui fini par 1 avec la même colonnes mais finissant par '2'


# Assuming 'dfraw' is your DataFrame and it's already loaded as in your provided code

def calculate_ratio(df):
    for col in df.columns:
        if (col.startswith('pfi') or col.startswith('pff')) and col.endswith('1'):
            rootc = col[:-1]
            print(rootc)
            col2 = rootc + '2'  # Construct the corresponding column name ending with '2'
            if col2 in df.columns:
                df[f'{rootc}_ratio'] = df[col] / (df[col] + df[col2])
                df[f'win_{rootc}'] = (df[f'{rootc}_ratio'] > 0.5).astype(int)
                print(df[df[f'win_{rootc}'] == 1].tgf_win_1.value_counts())
            else:
                print(f"Warning: Column '{col2}' not found for '{col}'. Ratio calculation skipped.")
    return df

dfraw = calculate_ratio(dfraw)


In [ ]:
df['win_nbv'] = (df.pfi_dNbVictoires > 0.5).astype(int)
df['win_gain'] = (df.pff_dGain > 0.5).astype(int)

In [ ]:
df[df.win_gain == 1].tgf_win_1.value_counts()

In [ ]:
df[df.win_nbv == 0].tgf_win_1.value_counts()

In [ ]:
# prompt: je veux la première moitié des enregistrements de test

# Get the first half of the test dataset
half_test = int(len(test) / 2)
first_half_test = test.head(half_test)

# Now you can work with the first_half_test DataFrame
print(len(first_half_test), 'first half of test examples')
# Example: Print the first few rows
print(first_half_test.head())


In [ ]:
half_test = int(len(test) / 2)
train0 = test.head(half_test)
train1 = test.tail(half_test)
print(len(train0[(train0.win_ord == 1) & (train0.win1 == 1)]) / len(train0[(train0.win1 == 1)]))
print(len(train1[(train1.win_ord == 1) & (train1.win1 == 1)]) / len(train1[(train1.win1 == 1)]))

In [ ]:
# prompt: je veux selectionner tout les enregistrements ou OS_N_SG_pis_cheval_1 est supérieur à sa valeur médiane

median_value = dfraw['OS_N_SG_pis_cheval_1'].median()
filtered_df = dfraw[dfraw['OS_N_SG_pis_cheval_1'] < median_value].copy()

# prompt: sur ce qui reste que je veux connaitre la valeur moyenne de OS_N_MU_pis_cheval_1 pour chaque quartile de pff_chGainTotal_1

# Calculate quartiles for pff_chGainTotal_1
filtered_df['quartile'] = pd.qcut(filtered_df['pff_chGainTotal_1'] / filtered_df['pfi_chNbCourses_1'], 4, labels=False)

# Group by quartile and calculate the mean of OS_N_MU_pis_cheval_1
mean_by_quartile = filtered_df.groupby('quartile')['OS_N_MU_pis_cheval_1'].mean()

mean_by_quartile

In [ ]:
# prompt: pour tout les enregistrement où pff_mu_pis_cheval_1 est = 25, je veux que la valeur soit remplacé par 24 si pff_chGainTotal_1 / pfi_chNbCourses_1 < 110000, par 29 si >110000 et <174000, par 32 si >174000 et <285000 et par 36 si > 285000

# Create a copy to avoid SettingWithCopyWarning
filtered_df = filtered_df.copy()

# Apply the conditions and update pff_mu_pis_cheval_1
filtered_df.loc[
    (filtered_df['pff_mu_pis_cheval_1'] == 25) &
    (filtered_df['pff_chGainTotal_1'] / filtered_df['pfi_chNbCourses_1'] < 110000),
    'pff_mu_pis_cheval_1'
] = 24

filtered_df.loc[
    (filtered_df['pff_mu_pis_cheval_1'] == 25) &
    (filtered_df['pff_chGainTotal_1'] / filtered_df['pfi_chNbCourses_1'] >= 110000) &
    (filtered_df['pff_chGainTotal_1'] / filtered_df['pfi_chNbCourses_1'] < 174000),
    'pff_mu_pis_cheval_1'
] = 29

filtered_df.loc[
    (filtered_df['pff_mu_pis_cheval_1'] == 25) &
    (filtered_df['pff_chGainTotal_1'] / filtered_df['pfi_chNbCourses_1'] >= 174000) &
    (filtered_df['pff_chGainTotal_1'] / filtered_df['pfi_chNbCourses_1'] < 285000),
    'pff_mu_pis_cheval_1'
] = 32

filtered_df.loc[
    (filtered_df['pff_mu_pis_cheval_1'] == 25) &
    (filtered_df['pff_chGainTotal_1'] / filtered_df['pfi_chNbCourses_1'] >= 285000),
    'pff_mu_pis_cheval_1'
] = 36

In [ ]:
# prompt: je veux connnaitre les bornes des quartiles de pff_chGainTotal_1 / pfi_chNbCourses_1

# Calculate quartiles for pff_chGainTotal_1 / pfi_chNbCourses_1
filtered_df['quartile_ratio'] = pd.qcut(filtered_df['pff_chGainTotal_1'] / filtered_df['pfi_chNbCourses_1'], 4, labels=False)

# Get the boundaries of each quartile
quartiles = filtered_df['pff_chGainTotal_1'] / filtered_df['pfi_chNbCourses_1']
quartile_bounds = quartiles.quantile([0.25, 0.5, 0.75])

quartile_bounds

In [ ]:
df = df[df.tgf_win_1 != 0.5]
print(len(df[(df.pfs_dSexe == 'FF') & (df.tgf_win_1 == 1)]) / len(df[(df.pfs_dSexe == 'FF')]))
print(len(df[(df.pfs_dSexe == 'FM') & (df.tgf_win_1 == 1)]) / len(df[(df.pfs_dSexe == 'FM')]))
print(len(df[(df.pfs_dSexe == 'FH') & (df.tgf_win_1 == 1)]) / len(df[(df.pfs_dSexe == 'FH')]))
print(len(df[(df.pfs_dSexe == 'HF') & (df.tgf_win_1 == 1)]) / len(df[(df.pfs_dSexe == 'HF')]))
print(len(df[(df.pfs_dSexe == 'HM') & (df.tgf_win_1 == 1)]) / len(df[(df.pfs_dSexe == 'HM')]))
print(len(df[(df.pfs_dSexe == 'HH') & (df.tgf_win_1 == 1)]) / len(df[(df.pfs_dSexe == 'HH')]))
print(len(df[(df.pfs_dSexe == 'MM') & (df.tgf_win_1 == 1)]) / len(df[(df.pfs_dSexe == 'MM')]))
print(len(df[(df.pfs_dSexe == 'MF') & (df.tgf_win_1 == 1)]) / len(df[(df.pfs_dSexe == 'MF')]))
print(len(df[(df.pfs_dSexe == 'MH') & (df.tgf_win_1 == 1)]) / len(df[(df.pfs_dSexe == 'MH')]))

In [ ]:
[(train_features, label_batch)] = train_ds.take(1)
test_type_col = train_features['diff_sexe']
test_type_layer = get_category_encoding_layer(name='diff_sexe',
                                              dataset=train_ds,
                                              dtype='string',
                                              max_tokens=5)
test_type_layer(test_type_col)

In [ ]:
model.save('classifier_test2.keras')
reloaded_model = tf.keras.models.load_model('classifier_test.keras')

In [ ]:
predictions = model.predict(test_ds)
predictions

In [ ]:
# prompt: je veux mettre le résultat des prédictions dans une colonne 'pred' du dataframe d'origine 'test'

# Convert predictions to a binary classification (e.g., using a threshold)
threshold = 0.0
binary_predictions = (predictions > threshold).astype(int)

# Add the predictions as a new column 'pred' to the test dataframe
test['pred'] = binary_predictions

# Print the updated dataframe with predictions
test.pred.value_counts()

In [ ]:
test.tgf_win_1.value_counts()

In [ ]:
# prompt: dans dataframe 'test' je veux une colonne win_cote qui contient 1 si rapport 2 > rapport 1 et 0 sinon

test['win_cote'] = (test['diff_cote'] < 0.5).astype(int)
test['win_ord'] = (test['diff_ord'] > 0.5).astype(int)

In [ ]:
test

In [ ]:
print(len(test[(test.pred == 1) & (test.tgf_win_1 == 1)]) / len(test[(test.tgf_win_1 == 1)]))
print(len(test[(test.win_cote == 1) & (test.tgf_win_1 == 1)]) / len(test[(test.tgf_win_1 == 1)]))
print(len(test[(test.win_ord == 1) & (test.tgf_win_1 == 1)]) / len(test[(test.tgf_win_1 == 1)]))
print(len(test[(test.win_cote == 1) & (test.win_ord == 1) & (test.tgf_win_1 == 1)]) / len(test[(test.tgf_win_1 == 1)]))

In [ ]:
print(len(test[(test.pred == 0) & (test.tgf_win_1 == 0)]) / len(test[(test.tgf_win_1 == 0)]))
print(len(test[(test.win_cote == 0) & (test.tgf_win_1 == 0)]) / len(test[(test.tgf_win_1 == 0)]))
print(len(test[(test.win_ord == 0) & (test.tgf_win_1 == 0)]) / len(test[(test.tgf_win_1 == 0)]))

In [ ]:
print(len(test[(test.pred == 1) & (test.tgf_win_1 == 1)]) / len(test[(test.pred == 1)]))
print(len(test[(test.pred2 == 1) & (test.tgf_win_1 == 1)]) / len(test[(test.pred2 == 1)]))
print(len(test[(test.win_cote == 1) & (test.tgf_win_1 == 1)]) / len(test[(test.win_cote == 1)]))
print(len(test[(test.win_ord == 1) & (test.tgf_win_1 == 1)]) / len(test[(test.win_ord == 1)]))
print(len(test[(test.win_cote == 1) & (test.win_ord == 1) & (test.tgf_win_1 == 1)]) / len(test[(test.win_cote == 1) & (test.win_ord == 1)]))
print(len(test[(test.win_cote == 0) & (test.pred == 1) & (test.tgf_win_1 == 1)]) / len(test[(test.win_cote == 0) & (test.pred == 1)]))